In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import io
import datetime
pd.options.display.max_columns=999

In [24]:
# 2016
# date1 = '2016-04-02'
# date2 = '2016-10-02'
# 2017
date1 = '2017-04-02'
date2 = '2017-10-01'
# 2018
# date1 = '2018-03-29'
# date2 = '2018-10-01'
mydates = pd.date_range(date1, date2).tolist()
BASE_URL="http://rotoguru1.com/cgi-bin/byday.pl?game=dk&month=MONTH&day=DAY&year=YEAR" #Draftkings

In [25]:
positions_dict = dict()
positions_dict["2"] = "C"
positions_dict["3"] = "1B"
positions_dict["4"] = "2B"
positions_dict["5"] = "3B"
positions_dict["6"] = "SS"
positions_dict["7"] = "OF"

In [47]:
# for date in mydates:
result = pd.to_datetime(date)
result += datetime.timedelta(days=1)
date_str=str(result.month) + "_" + str(result.day) + "_" + str(result.year)
BASE_URL="http://rotoguru1.com/cgi-bin/byday.pl?game=dk&month=MONTH&day=DAY&year=YEAR"
URL = BASE_URL.replace("DAY", str(date.day)).replace("MONTH", str(date.month)).replace("YEAR", str(date.year))
context = requests.get(URL).content
soup = BeautifulSoup(context,'html.parser')
tables  = soup.find_all("tr")
len(tables)
print(tables[66].prettify())

<tr>
 <td>
  P
 </td>
 <td nowrap="">
  <a href="http://rotoguru1.com/cgi-bin/player17.cgi?109yx" target="_blank">
   Kelvin Herrera
  </a>
 </td>
 <td align="center">
  4.25
 </td>
 <td align="right">
  $4,200
 </td>
 <td align="center">
  KAN
 </td>
 <td nowrap="">
  v laa
 </td>
 <td nowrap="">
  3-2
 </td>
 <td align="left" nowrap="">
  1.0IP K
  <font color="green">
   Save
  </font>
 </td>
</tr>



In [50]:
parts = tables[66].find_all("td")

In [54]:
for part in parts:
    print(part)

<td>P</td>
<td nowrap=""><a href="http://rotoguru1.com/cgi-bin/player17.cgi?109yx" target="_blank">Kelvin Herrera</a></td>
<td align="center">  4.25</td>
<td align="right">$4,200</td>
<td align="center">KAN</td>
<td nowrap="">v laa</td>
<td nowrap=""> 3-2</td>
<td align="left" nowrap="">  1.0IP K <font color="green">Save</font> </td>
